In [1]:
import re
import requests
from bs4 import BeautifulSoup
import networkx as nx
import ipywidgets as widgets

G = nx.Graph()

In [2]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: C:\Users\Lyx\miniconda3\etc\jupyter
    - Validating...
      voila 0.2.16 ok


In [3]:
page = requests.get("http://dtu-graph.lyxx.fr/data/class.html")
soup = BeautifulSoup(page.content, "html.parser")

for i in soup.findAll("tr"):
    #fetch departements
    if "colspan" in (str)(list(i.children)[1]):
        departement = list(i.children)[1].text.strip()
    #fetch class
    else:
        classnumber = i.text.strip().split('\n')[0][:5]
        classname = i.text.strip().split('\n')[0][8:]
        langage, ects = [s.strip() for s in i.text.strip().split('\n')[1].split('|')[0:2]]
        ects = (float)(ects.split(' ')[0])

        #get schedule. First dimension : different option, second dimension : all the class to follow for this option
        res = ""
        for j in list(i.children)[3].findAll("span")[1:]:
            if j.text.strip() != "":
                res += j.text.strip()
        res = res.split("or")
        for j in range(len(res)):
            group = re.findall("([EF][1-5]) ", res[j])
            res[j] = re.findall("(January|June|July|August|Spring|[EF][1-5][AB])", res[j])
            for k in group:
                res[j].append(k + 'A')
                res[j].append(k + 'B')              
        G.add_node(classnumber, name=classname, langage=langage, ects=ects, departement=departement, date=res)

In [4]:
page = requests.get("http://dtu-graph.lyxx.fr/data/ranking.html")
soup = BeautifulSoup(page.content, "html.parser")

#fetch class ratings
for i in soup.findChild("tbody").findAll("tr"):
    classnum = list(i.children)[0].text
    #some class might not exist anymore
    try:
        G.nodes[classnum]["avgGrade"] = list(i.children)[2].text
        G.nodes[classnum]["passed"] = list(i.children)[4].text
        G.nodes[classnum]["rating"] = list(i.children)[5].text
        G.nodes[classnum]["workload"] = list(i.children)[6].text
    except:pass
    

In [5]:
#this sections checks if 2 class can happen at the same time
f = list(G.nodes())
def checkVariante(i, j):
    for iDate in G.nodes[f[i]]["date"]:
        for jDate in G.nodes[f[j]]["date"]:
            if not any(item in iDate for item in jDate):
                return True
    return False

In [6]:
#add edge to class that can happen at the same time
for i in range(len(G.nodes()) - 1):
    for j in range(i + 1, len(G.nodes())):
        if checkVariante(i, j):
            G.add_edge(f[i], f[j])

In [7]:
#Now preparing the filters for the UI.
langages = []
departements = []

for i in G.nodes():
    if not G.nodes[i]["departement"] in departements:
        departements.append(G.nodes[i]["departement"])
    if not G.nodes[i]["langage"] in langages:
        langages.append(G.nodes[i]["langage"])

In [8]:
#this function will be called almost at every event on the UI
def updateSelect(c):
    select.options = showPossibleClass()
    chosen.options = showChosen()
    labelEcts.value = "Total : " + str(ectsCounter()) + " ECTS"

In [9]:
#langage selection
selectLangages = widgets.SelectMultiple(
    options=langages,
    description='Langage',
    layout= widgets.Layout(width='auto')
)

selectLangages.observe(updateSelect, 'value')

In [10]:
#departement selection
selectDepartement = widgets.SelectMultiple(
    options=departements,
    description='Departement',
    layout= widgets.Layout(width='auto')
)

selectDepartement.observe(updateSelect, 'value')

In [11]:
labelEcts = widgets.Label()

In [12]:
#compute what should be displayed where
chosenList = []

def canAdd(node, takenSlot):
    for j in node:
        isPossible = True
        for k in j:
            if k in takenSlot:
                isPossible = False
        if isPossible:
            return True
    return False

def possibleClass():
    takenSlot, selectable = [], []
    for i in chosenList:
        for j in i["date"]:#TODO handle "or"
            for k in j:
                takenSlot.append(k)

    for i in G.nodes():
        if canAdd(G.nodes[i]["date"], takenSlot):
            selectable.append(G.nodes[i])
    return selectable
      
def classFiltered(selectable):
    selectableFiltered = []
    for i in selectable:
        if i["departement"] in selectDepartement.value and i["langage"] in selectLangages.value:
            selectableFiltered.append(i)
    return selectableFiltered

def showPossibleClass():
    display = []
    for i in classFiltered(possibleClass()):
        display.append(i["name"])
    return display

def showChosen():
    display = []
    for i in chosenList:
        display.append(i["name"])
    return display

def ectsCounter():
    res = 0
    for i in chosenList:
        res += i["ects"]
    return '%g'%(res)

In [13]:
#initiate the list of the chosen class, and the possible picks
select = widgets.Select(
    options=showPossibleClass(),
    description='Possible picks',
    layout= widgets.Layout(width='50%')
)

chosen = widgets.Select(
    description='Your picks',
    layout= widgets.Layout(width='50%')
)

#add reviews about the currently selected pick
labelClassRanking = widgets.Label()
def classRanking(c):
    for i in G.nodes():
        if G.nodes[i]["name"] == select.value:
            try:
                labelClassRanking.value = "Average grade : " + G.nodes[i]["avgGrade"] + " | " + \
                    "Passed : " + G.nodes[i]["passed"] + "% | " + "Rating : " + G.nodes[i]["rating"] + \
                    " | " + "Workload : " + G.nodes[i]["workload"] + "."
            except:
                labelClassRanking.value = "No reviews yet for this class."
            break
select.observe(classRanking, 'value')


In [14]:
#here we handle the buttons behaviour
def onAdd(c):
    for i in G.nodes():
        if G.nodes[i]["name"] == select.value:
            chosenList.append(G.nodes[i])
            break
    btnDel.disabled = False
    updateSelect(None)

def onRemove(c):
    for i in range(len(chosenList)):
        if chosenList[i]["name"] == chosen.value:
            chosenList.pop(i)
            break
    if len(chosenList) == 0:
        btnDel.disabled = True
    updateSelect(None)

btnAdd = widgets.Button(
    description='Add',
    button_style='success',
)

btnDel = widgets.Button(
    description='Remove',
    button_style='danger',
    disabled=True,
)
btnDel.on_click(onRemove)
btnAdd.on_click(onAdd)


In [15]:
widgets.VBox([
    widgets.HBox([chosen, btnDel]),
    labelEcts,
    widgets.Label("Filters"),
    widgets.HBox([selectDepartement, selectLangages]),
    widgets.HBox([select, btnAdd]),
    labelClassRanking   
])
